# Семинар 10. Стиль кодирования. Юнит-тестирование.

## Стиль кодирования pep8

Полный текст есть в документации: https://www.python.org/dev/peps/pep-0008/
По сути, представляет собой восьмое предложение по «расширению» Python (Python Enhancement Proposal, PEP).

### Внешний вид кода

* отступ — 4 пробела
* не более 79 символов на строку
* определение классов и отельных функций отбиваются двойными новыми строками
* определения методов в классе отбиваются одинарными новыми строками
* код в UTF-8
* каждый `import` на отдельной строчке
* следует избегать конструкций вида `import *`
* порядок импортирования: модули из стандартной библиотеки, библиотечные модули, модули конкретного проекта

**Правильно**:

    foo = long_function_name(var_one, var_two,
                             var_three, var_four)

или

    def long_function_name(
            var_one, var_two, var_three,
            var_four):
        print(var_one)


или

    foo = long_function_name(
        var_one, var_two,
        var_three, var_four)

**Направильно**:

    foo = long_function_name(var_one, var_two,
        var_three, var_four)

или

    def long_function_name(
        var_one, var_two, var_three,
        var_four):
        print(var_one)

### Кавычки и пробелы

Следует использовать одни и те же (одиночные или двойные) кавычки внутри одного файла. Для тройных кавычек всегда следует использовать двойные кавычки.

Пробелы следует ставить для повышения читаемости, но не слишком много. Например, это неправильно:

    spam( ham[ 1 ], { eggs: 2 } )
    ham[1: 9], ham[1 :9], ham[1:9 :3]
    dct ['key'] = lst [index]
    if x == 4 : print x , y ; x , y = y , x
    i=i+1

А это правильно:

    spam(ham[1], {eggs: 2})
    if x == 4: print x, y; x, y = y, x
    ham[1:9], ham[1:9:3], ham[:9:3], ham[1::3], ham[1:9:]
    dct['key'] = lst[index]
    i = i + 1
    
Висящие пробелы запрещены.

### Соглашения об именовании

Полный текст: https://www.python.org/dev/peps/pep-0008/

Особые случаи:
* \_single_leading_underscore — «слабый» идентификатор внутреннего использования
* single\_trailing\_underscore\_ — используются, чтобы избегать конфликтов с именами, уже определенными в Python
* \_\_double_leading_underscore — сильный идентификатор внутреннего использования, в случае аттрибутов класса вызывает преобразование имен
* \_\_foo\_bar\_\_ — «магические» объекты, новые лучше не создавать, только переопределять существующие

Основные правила:
* имена модулей должны быть записаны только в нижнем регистре
* ИменаКлассов в CamelCase
* в именах исключений должен быть суффикс «Error»
* имена\_функций должны быть в нижнем регистре с подчеркиваниями
* константы именуются в верхнем регистре с подчеркиваниями: MAX_VALUE

### Рекомендации по написанию кода

* внимание к публичным и приватным членам классов
* сравнение с синглтонами типа `None` всегда долны быть через «is» или «is not», и осторожнее с булевыми значениями
* «is not» лучше, чем «not ... is»

## Модуль unittest

Базовая единица в тестировании — тестовые случаи (Test Cases). Для того, чтобы создать тестовый случай в `unittest`, необходимо определить класс и наследоваться от `unittest.TestCase` или `unittest.FunctionTestCase`. Тестовый случай должен быть абсолютно независимым.

Важные методы:
  * setUp() — метод, описывающий необходимую подготовку до запуска тестового случая
  * tearDown() — запускается после выполнения всех тестов тестового случая
  * остальные: https://docs.python.org/3.6/library/unittest.html#test-cases
  
Варианты assert'ов:
  * assertEqual(a, b) проверяет `a == b`	 
  * assertNotEqual(a, b) проверяет `a != b`
  * assertTrue(x) проверяет bool(x) `is True`
  * assertFalse(x) проверяет bool(x) `is False` 
  * assertIs(a, b) проверяет `a is b`
  * assertIsNot(a, b) проверяет `a is not b`
  * assertIsNone(x) проверяет `x is None`
  * assertIsNotNone(x) проверяет `x is not None`
  * assertIn(a, b) проверяет `a in b`
  * assertNotIn(a, b) проверяет `a not in b`
  * assertIsInstance(a, b) проверяет `isinstance(a, b)`
  * assertNotIsInstance(a, b) проверяет `not isinstance(a, b)`

В assert можно добавить последний аргумент msg — сообщение, которое будет выведено в случае падения теста.

In [57]:
%%writefile mymodule.py
def sum_two_numbers(a, b):
    return a + b

Overwriting mymodule.py


In [58]:
%%writefile test_mymodule.py
#!/usr/bin/env python
import unittest
import mymodule

class TestMyModule(unittest.TestCase):

    def test_sum_two_numbers1(self):
        res = mymodule.sum_two_numbers(1, 3)
        self.assertEqual(res, 4)
    
    def test_sum_two_numbers2(self):
        res = mymodule.sum_two_numbers(1, 3)
        self.assertNotEqual(res, 5)

    def test_sum_two_numbers3(self):
        with self.assertRaises(TypeError):
            mymodule.sum_two_numbers(1, None)

if __name__ == '__main__':
    unittest.main()

Overwriting test_mymodule.py


In [59]:
%%bash

python test_mymodule.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK


In [40]:
%%bash

python -m unittest test_mymodule.TestMyModule.test_sum_two_numbers1

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [60]:
%%bash

python -m unittest discover

...
----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK


## Модуль doctest

Подробнее: https://docs.python.org/3.5/library/doctest.html

In [64]:
%%writefile doctstex.py

"""
This is the "example" module.

The example module supplies one function, factorial().  For example,

>>> factorial(5)
120
"""

def factorial(n):
    """Return the factorial of n, an exact integer >= 0.

    If the result is small enough to fit in an int, return an int.
    Else return a long.

    >>> [factorial(n) for n in range(6)]
    [1, 1, 2, 6, 24, 120]
    >>> [factorial(int(n)) for n in range(6)]
    [1, 1, 2, 6, 24, 120]
    >>> factorial(29)
    265252859812191058636308480000000
    >>> factorial(30)
    265252859812191058636308480000000
    >>> factorial(-1)
    Traceback (most recent call last):
        ...
    ValueError: n must be >= 0

    Factorials of floats are OK, but the float must be an exact integer:
    >>> factorial(30.1)
    Traceback (most recent call last):
        ...
    ValueError: n must be exact integer
    >>> factorial(30.0)
    265252859812191058636308480000000

    It must also not be ridiculously large:
    >>> factorial(1e100)
    Traceback (most recent call last):
        ...
    OverflowError: n too large
    """

    import math
    if not n >= 0:
        raise ValueError("n must be >= 0")
    if math.floor(n) != n:
        raise ValueError("n must be exact integer")
    if n+1 == n:  # catch a value like 1e300
        raise OverflowError("n too large")
    result = 1
    factor = 2
    while factor <= n:
        result *= factor
        factor += 1
    return result


if __name__ == "__main__":
    import doctest
    doctest.testmod()

Overwriting doctstex.py


In [65]:
%%bash

python doctstex.py

**********************************************************************
File "doctstex.py", line 21, in __main__.factorial
Failed example:
    factorial(29)
Expected:
    265252859812191058636308480000000
Got:
    8841761993739701954543616000000
**********************************************************************
1 items had failures:
   1 of   8 in __main__.factorial
***Test Failed*** 1 failures.


In [66]:
%%bash

python doctstex.py -v

Trying:
    factorial(5)
Expecting:
    120
ok
Trying:
    [factorial(n) for n in range(6)]
Expecting:
    [1, 1, 2, 6, 24, 120]
ok
Trying:
    [factorial(int(n)) for n in range(6)]
Expecting:
    [1, 1, 2, 6, 24, 120]
ok
Trying:
    factorial(29)
Expecting:
    265252859812191058636308480000000
**********************************************************************
File "doctstex.py", line 21, in __main__.factorial
Failed example:
    factorial(29)
Expected:
    265252859812191058636308480000000
Got:
    8841761993739701954543616000000
Trying:
    factorial(30)
Expecting:
    265252859812191058636308480000000
ok
Trying:
    factorial(-1)
Expecting:
    Traceback (most recent call last):
        ...
    ValueError: n must be >= 0
ok
Trying:
    factorial(30.1)
Expecting:
    Traceback (most recent call last):
        ...
    ValueError: n must be exact integer
ok
Trying:
    factorial(30.0)
Expecting:
    265252859812191058636308480000000
ok
Trying:
    factorial(1e100)
Expecting:
    T